In [1]:
import numpy as np
from sklearn.cluster import DBSCAN

class IRObjectDetector:
    def __init__(self, num_sensors, max_distance, eps=0.5, min_samples=3):
        self.num_sensors = num_sensors
        self.max_distance = max_distance
        self.eps = eps
        self.min_samples = min_samples
        self.dbscan = DBSCAN(eps=eps, min_samples=min_samples)

    def detect_objects(self, ir_data):
        # Preprocess IR data
        ir_data = np.array(ir_data) / self.max_distance  # Normalize distances
        ir_data = np.clip(ir_data, 0, 1)  # Clip values between 0 and 1

        # Create feature vectors
        angles = np.linspace(0, 2 * np.pi, self.num_sensors, endpoint=False)
        features = np.column_stack((ir_data * np.cos(angles), ir_data * np.sin(angles)))

        # Cluster points using DBSCAN
        clusters = self.dbscan.fit_predict(features)

        # Extract objects from clusters
        objects = []
        for cluster_id in np.unique(clusters):
            if cluster_id != -1:
                cluster_points = features[clusters == cluster_id]
                center = np.mean(cluster_points, axis=0)
                radius = np.max(np.linalg.norm(cluster_points - center, axis=1))
                objects.append({'center': center, 'radius': radius})

        return objects

In [2]:
# Create an instance of the IRObjectDetector
detector = IRObjectDetector(num_sensors=8, max_distance=5.0)

# Sample IR sensor data
ir_data = [2.5, 1.8, 1.2, 0.9, 1.1, 1.5, 2.1, 3.0]

# Detect objects
objects = detector.detect_objects(ir_data)

# Print detected objects
for obj in objects:
    print(f"Object center: {obj['center']}, radius: {obj['radius']}")

Object center: [ 0.07742641 -0.05431981], radius: 0.5071046458188039
